# BERT tanítása ONNX Runtime környezetben
A notebook végigvezet a futtatás és tanítás lépésein AzureML környezetben. Ajánlatott az alábbi, általunk készített angol nyelvű [cikk](https://towardsdatascience.com/train-bert-large-in-your-own-language-7685ee26b05b) használata a futtatás során, azon felül, hogy lehetőségek szerint megpróbáljuk a lehető legbővebben és legértelmezhetőbben leírni a futtatás és tanítás menetét. A notebook futtatásához egy AzureML compute instance szükséges, mi ehhez a STANDARD_DS1_V2 virtuális gépet ajánljuk.

Lépések:
- AzureML Workspace betöltése
- BLOB tároló regisztrálása
- AzureML experiment készítése
- Compute target (virtuális gép) készítése
- Estimator készítése a futtatáshoz
- Konfiguráció és futtatás

Hasznos linkek:
[Compute Instance](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-instance)
[Estimator használata AzureML-ben](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training-with-deep-learning/how-to-use-estimator/how-to-use-estimator.ipynb)
[ONNX Runtime BERT](https://github.com/microsoft/onnxruntime-training-examples/blob/master/nvidia-bert/README.md)

### AzureML SDK betöltése

In [ ]:
import os
import requests
import sys

# AzureML könyvtárak importálása
import azureml.core
from azureml.core import Experiment, Workspace, Datastore, Run
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.container_registry import ContainerRegistry
from azureml.core.runconfig import MpiConfiguration, RunConfiguration, DEFAULT_GPU_IMAGE
from azureml.train.dnn import PyTorch
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

# AzureML SDK verzió
print("SDK version:", azureml.core.VERSION)

### AzureML Workspace beállítása

In [ ]:
# AzureML workspace beállítása
# info: https://docs.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py
# illetve, https://towardsdatascience.com/train-bert-large-in-your-own-language-7685ee26b05b
ws = Workspace.get(name="myworkspace", subscription_id='<azure-subscription-id>', resource_group='myresourcegroup')

# Workspace attribútumok kiíratása
print('Workspace name: ' + ws.name, 
      'Workspace region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

### BLOB tároló regisztrálása
Az adathalmaz előkészítésekor BLOB tárolóba mozgattuk a kész bináris fájlokat, itt szükséges regisztálni a BLOB tárolót, amivel később hozzá tudunk férni a feltöltött fájlokhoz. Itt javasoljuk az Azure Storage Explorer használatát (https://azure.microsoft.com/en-us/features/storage-explorer/), illetve https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data ezen oldal átnézését.

In [ ]:
# Regisztráljuk adattárolóként a BLOB adathalmazt.
ds = Datastore.register_azure_blob_container(workspace=ws, 
                                             datastore_name='<datastore-name>',
                                             account_name='<storage-account-name>', 
                                             account_key='<storage-account-key>',
                                             container_name='<storage-container-name>')

In [ ]:
# Datastore attribútumok kiíratása
print('Datastore name: ' + ds.name, 
      'Container name: ' + ds.container_name, 
      'Datastore type: ' + ds.datastore_type, 
      'Workspace name: ' + ds.workspace.name, sep = '\n')

### AzureML Compute Cluster készítése
Ezzel a pár sor kóddal AzureML Compute Cluster-t tudunk létrehozni, amin a tanítást futtatni tudjuk. Itt érdemes a kvótáknak megfelelő eszközöket használni, mi a futtatásra Standard_NC24rs_v3 vagy Standard_ND40rs_v2 gépet ajánljuk. Egy Standard_NC24rs_v3 klaszteren (virtuális gép) a futtatás körülbelül 200 órát vesz igénybe.

In [ ]:
# GPU klaszter készítése
gpu_cluster_name = "ortbertpretrain" 
try:
    gpu_compute_target = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print('GPU klaszter már létezik.')
except ComputeTargetException:
    print('Új GPU klaszter készítése...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_ND24rs_v3', min_nodes=0, max_nodes=8)
    gpu_compute_target = ComputeTarget.create(ws, gpu_cluster_name, compute_config)
    gpu_compute_target.wait_for_completion(show_output=True)

In [ ]:
# Experiment készítése a futtatáshoz:
# AzureML-ben a nagyméretű futtatások experimentként vannak jegyezve, így az AzureML SDK-val könnyedén létre tudunk hozni egy új "kísérletet", amivel futtatni tudjuk a tanításunkat.
experiment_name = 'nvbert-ort-pretraining-phase1'
experiment = Experiment(ws, name=experiment_name)

### Estimator készítése
Az AzureML Estimator az az eszköz, amivel el tudunk indítani egy új kísérletet. Itt tudjuk beállítani a hyperparamétereket is a futtatandó tanításhoz. Az alábbi táblázat segítséget nyújt a batch méret beállításához:

| VM SKU             | node_count         | gpu_memory_limit_gb         | train_batch_size | gradient_accumulation_steps |
| ------------------ |:------------------:|-----------------:|-----------------:| ---------------------------:|
| Standard_ND40rs_v2 | 1 (8 GPUs total)   | 32  | 8192  | 64  |
| Standard_ND40rs_v2 | 2 (16 GPUs total)  | 32  | 4096  | 32  |
| Standard_ND40rs_v2 | 4 (32 GPUs total)  | 32  | 2048  | 16  |
| Standard_ND40rs_v2 | 8 (64 GPUs total)  | 32  | 1024  | 8   |
| Standard_NC24rs_v3 | 1 (4 GPUs total)   | 16  | 16320 | 340 |
| Standard_NC24rs_v3 | 2 (8 GPUs total)   | 16  | 8160  | 170 |
| Standard_NC24rs_v3 | 4 (16 GPUs total)  | 16  | 4080  | 85  |
| Standard_NC24rs_v3 | 8 (32 GPUs total)  | 16  | 2016  | 42  |
| Standard_NC24rs_v3 | 16 (64 GPUs total) | 16  | 1008  | 21  |

Több infó található erről az eredeti [README.md](../README.md) fájlban.

In [ ]:
# ez a változó a projekt könyvtára, ahol a tanító szkriptek találhatóak
project_folder = '../../workspace/BERT'

# előkészített Docker képfájl, ami a Microsoft ONNX Runtime mérnökei által lett elkészítve, az AzureML platform automatikusan ezt a képfájl fogja használni a számítási klaszteren, így ez tartalmazza a számunkra szükséges eszközöket, mint a Python könytárak, vagy NVIDIA csomagok, mint a CUDA vagy CUDNN.
image_name = 'mcr.microsoft.com/azureml/onnxruntime-training:0.1-rc3.1-openmpi4.0-cuda10.2-cudnn8.0-nccl2.7-for-bert'

# MPI konfiguráció
# Node-onkénti GPU-k számát kell itt beállítani Standard_NC24rs_v3 esetén 4, Standard_ND40rs_v2 esetén 8.
mpi = MpiConfiguration()
mpi.process_count_per_node = 4

import uuid
output_id = uuid.uuid1().hex

# Első fázis Estimator készítése
estimator_ph1 = Estimator(source_directory=project_folder,

                    # Számítási konfiguráció
                    compute_target = gpu_compute_target,
                    node_count=4,
                    process_count_per_node=1,  
                    distributed_training = mpi,
                    use_gpu = True,
                    
                    # Docker képfájl betöltése
                    use_docker = True,
                    custom_docker_image = image_name,
                    user_managed = True,
                    
                    # Tanító szkript paraméterek
                    # A batch méret és gradient accumulation steps, illetve gpu_memory_limit_gb paraméterekhez a fenti táblázat nyújt segítséget!
                    script_params = {
                        "--config_file": "bert_config.json", # általunk kívánt BERT config, small vagy base vagy large, esetleg kisebb modell.
                        '--input_dir' : ds.path('<blob-path-to-phase1-training-data>').as_mount(), # első fázis tanító adathalmaz elérés megadása.
                        '--output_dir': ds.path(f'output/{experiment_name}/{output_id}/').as_mount(),
                        '--bert_model' : 'bert-large-uncased',
                        '--train_batch_size' : 2048,
                        '--max_seq_length': 128,
                        '--max_predictions_per_seq': 20,
                        '--max_steps' : 7038,
                        '--warmup_proportion' : '0.2843',
                        '--num_steps_per_checkpoint' : 200,
                        '--learning_rate' : '6e-3',
                        '--seed': 42,
                        '--fp16' : '',
                        '--gradient_accumulation_steps' : 16,
                        '--allreduce_post_accumulation' : '',
                        '--allreduce_post_accumulation_fp16' : '',
                        '--do_train' : '',
                        '--use_ib' : '', 
                        '--gpu_memory_limit_gb' : 16
                    },
                    
                    entry_script = 'run_pretraining_ort.py',
                    inputs = [ds.path('<blob-path-to-phase1-training-data>').as_mount()]
                   )

### AzureML Experiment futtatása - Első fázis

In [ ]:
# Első fázis futtatása (ha ez lefut, akkor az Experiments oldalunk tudjuk a futtatást, logokat megnézni)
run = experiment.submit(estimator_ph1)
RunDetails(run).show()
print(run.get_portal_url())

In [ ]:
# Experiment készítése a második fázishoz, ha az első lefutott:
experiment_name = 'nvbert-ort-pretraining-phase2'
experiment = Experiment(ws, name=experiment_name)

### Estimator készítése - Második fázis
Ha az első fázis lefutott, akkor a második fázis következik. A szükséges paraméterekhez a lenti táblázat nyújt segítséget és emelett ne felejtsük el a második fázis adathalmazát betölteni a BLOB tárolóból.

| VM SKU             | node_count         | gpu_memory_limit_gb         | train_batch_size | gradient_accumulation_steps |
| ------------------ |:------------------:|-----------------:|-----------------:| ---------------------------:|
| Standard_ND40rs_v2 | 1 (8 GPUs total)   | 32  | 4096 | 256  |
| Standard_ND40rs_v2 | 2 (16 GPUs total)  | 32  | 2048 | 128  |
| Standard_ND40rs_v2 | 4 (32 GPUs total)  | 32  | 1024 | 64   |
| Standard_ND40rs_v2 | 8 (64 GPUs total)  | 32  | 512  | 32   |
| Standard_NC24rs_v3 | 1 (4 GPUs total)   | 16  | 8192 | 1024 |
| Standard_NC24rs_v3 | 2 (8 GPUs total)   | 16  | 4096 | 512  |
| Standard_NC24rs_v3 | 4 (16 GPUs total)  | 16  | 2048 | 256  |
| Standard_NC24rs_v3 | 8 (32 GPUs total)  | 16  | 1024 | 128  |
| Standard_NC24rs_v3 | 16 (64 GPUs total) | 16  | 512  | 64   |

In [ ]:

# Estimator készítése a második fázishoz:
estimator_ph2 = Estimator(source_directory=project_folder,

                    # Számítási konfiguráció
                    compute_target = gpu_compute_target,
                    node_count=4, 
                    process_count_per_node=1, 
                    distributed_training = mpi,
                    use_gpu = True,
                    
                    # Docker képfájl betöltése
                    use_docker = True,
                    custom_docker_image = image_name,
                    user_managed = True,
                    
                    # Tanító szkript paraméterek
                    script_params = {
                        "--config_file": "bert_config.json",
                        '--input_dir' : ds.path('<blob-path-to-phase2-training-data>').as_mount(), 
                        '--output_dir': ds.path(f'output/{experiment_name}/{output_id}/').as_mount(),
                        '--bert_model' : 'bert-large-uncased',
                        '--train_batch_size' : 1024,
                        '--max_seq_length': 512,
                        '--max_predictions_per_seq': 80,
                        '--max_steps' : 1563,
                        '--warmup_proportion' : '0.128',
                        '--num_steps_per_checkpoint' : 200,
                        '--learning_rate' : '4e-3',
                        '--seed': 42,
                        '--fp16' : '',
                        '--gradient_accumulation_steps' : 64,
                        '--allreduce_post_accumulation' : '',
                        '--allreduce_post_accumulation_fp16' : '',
                        '--do_train' : '',
                        '--phase2' : '',
                        '--resume_from_checkpoint' : '',
                        '--phase1_end_step' : '7038',
                        '--init_checkpoint' : ds.path('<path-to-checkpoint-from-phase-1>'),
                        '--use_ib' : '', 
                        '--gpu_memory_limit_gb' : 16
                    },
                    
                    entry_script='run_pretraining_ort.py',
                    inputs=[ds.path('<blob-path-to-phase2-training-data>').as_mount()])

### AzureML Experiment futtatása - Második fázis

In [ ]:
# Második fázis futtatása (ha ez lefut, akkor az Experiments oldalunk tudjuk a futtatást, logokat megnézni)
run = experiment.submit(estimator_ph2)
RunDetails(run).show()
print(run.get_portal_url())